In [2]:
#import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


Things to do
- [x] just ask it what the outcome of the Purdue High Point game is gonna be
- [x] read Henderson's data into pandas dataframe
- [x] filter out just Purdue and High Point's (away) games
- [x] create a time series of each team's score (after each scoring event) for each half
- [x] put one game's first half spread/sum into you.com and predict the second half
- [x] do few shot by providing the last 10 games of each team, the first half spread/sum followed by the corresponding second half 
- [ ] add in a prompt, for a you.com second-half predictor, agent, that says to use available info on the web as a prior but to take the actual first half data into account
- [ ] get a list of all team names from the data file
- [ ] write a function that iterates over a list of names, generating all the files
- [ ] write a plotting function (perhaps borrow from deepseek notebook)
- [ ] create agent for the first half
- [ ] update agent prompt to focus on the data and give it a persona

---

**Q: What do you predict the score of the Purdue vs High Point NCAA Men's basketball game is going to be tomorrow?**

**Answer:** Based on the available data and implied odds from the spread and over/under, a reasonable score prediction for Purdue vs. High Point is Purdue 80, High Point 73.

In [29]:
# Read the data from the JSONL file into a pandas DataFrame
df_OG = pd.read_json('../data/season_2025.tweets.jsonl', lines=True)

In [ ]:
df_OG.head()

,game_id,half,secs_left_reg,secs_left_half,home_score,away_score,home_team,away_team,play_desc
0,401706962,1,2376,1176,0,0,Kansas Jayhawks,Howard Bison,Rylan Griffen missed Three Point Jumper.
1,401706962,1,2374,1174,0,0,Kansas Jayhawks,Howard Bison,Bryce Harris Defensive Rebound.
2,401706962,1,2359,1159,0,0,Kansas Jayhawks,Howard Bison,Blake Harper missed Layup.
3,401706962,1,2357,1157,0,0,Kansas Jayhawks,Howard Bison,KJ Adams Jr. Defensive Rebound.
4,401706962,1,2353,1153,3,0,Kansas Jayhawks,Howard Bison,David Coit made Three Point Jumper. Assisted b...


In [ ]:
#list of unique away_team values
unique_away_teams = df_OG['away_team'].unique()
sorted_unique_away_teams = sorted(unique_away_teams)
print(sorted_unique_away_teams)

['Abilene Christian Wildcats', 'Air Force Falcons', 'Akron Zips', 'Alabama A&M Bulldogs', 'Alabama Crimson Tide', 'Alabama State Hornets', 'Alcorn State Braves', 'Alice Lloyd College Eagles', 'American University Eagles', 'Anderson (IN) Ravens', 'App State Mountaineers', 'Apprentice School Builders', 'Aquinas Saints', 'Arizona State Sun Devils', 'Arizona Wildcats', 'Arkansas Baptist Buffaloes', 'Arkansas Razorbacks', 'Arkansas State Red Wolves', 'Arkansas-Pine Bluff Golden Lions', 'Arlington Baptist Patriots', 'Army Black Knights', 'Asbury Eagles', 'Auburn Tigers', 'Auburn-Montgomery Senators', 'Augusta Jaguars', 'Austin Peay Governors', 'Averett Cougars', 'Avila Eagles', 'BYU Cougars', 'Baldwin Wallace Yellow Jackets', 'Ball State Cardinals', 'Baylor Bears', 'Bellarmine Knights', 'Belmont Bruins', 'Benedictine (AZ) Redhawks', 'Benedictine University (IL) Eagles', 'Bethany (WV) Bison', 'Bethesda University Flames', 'Bethune-Cookman Wildcats', 'Binghamton Bearcats', "Bluefield Universit

In [125]:
substring = "Troy"

# Filter the list for items containing the substring
filtered_teams = [team for team in sorted_unique_away_teams if substring in team]

# Print the filtered list
print(filtered_teams)

['Troy Trojans']


In [126]:
thursday_teams = ['Auburn Tigers','Alabama State Hornets',
                  'Louisville Cardinals','Creighton Bluejays',
                  'Michigan Wolverines','UC San Diego Tritons',
                  'Texas A&M Aggies','Yale Bulldogs',
                  'Missouri Tigers','Drake Bulldogs',
                  'Texas Tech Red Raiders','UNC Wilmington Seahawks',
                  'Kansas Jayhawks','Arkansas Razorbacks',
                  "St. John's Red Storm",'Omaha Mavericks',
                  'BYU Cougars','VCU Rams',
                  'Wisconsin Badgers','Montana Grizzlies',
                  'Houston Cougars','SIU Edwardsville Cougars',
                  'Gonzaga Bulldogs','Georgia Bulldogs',
                  'Clemson Tigers','McNeese Cowboys',
                  'UCLA Bruins', 'Utah State Aggies',
                  'Tennessee Volunteers', 'Wofford Terriers'

                  ]

friday_teams = ['Ole Miss Rebels', 'North Carolina Tar Heels',
                'Iowa State Cyclones', 'Lipscomb Bisons',
                'Marquette Golden Eagles', 'New Mexico Lobos',
                'Michigan State Spartans', 'Bryant Bulldogs',
                'Florida Gators', 'Norfolk State Spartans',
                'UConn Huskies', 'Oklahoma Sooners',
                'Memphis Tigers', 'Colorado State Rams',
                'Maryland Terrapins', 'Grand Canyon Lopes',
                'Duke Blue Devils', "Mount St. Mary's Mountaineers",
                'Mississippi State Bulldogs', 'Baylor Bears',
                'Oregon Ducks', 'Liberty Flames',
                'Arizona Wildcats', 'Akron Zips',
                "Saint Mary's Gaels", 'Vanderbilt Commodores',
                'Alabama Crimson Tide', 'Robert Morris Colonials',
                'Illinois Fighting Illini', 'Xavier Musketeers',
                'Kentucky Wildcats', 'Troy Trojans'

                ]

In [ ]:
#select only those games that the Purdue Boilermakers played as the away_team
df_purdue_away = df_OG[df_OG['away_team'].str.contains('Purdue Boilermakers')]

df_HighPoint_away = df_OG[df_OG['away_team'].str.contains('High Point Panthers')]
[len(df_purdue_away), len(df_HighPoint_away)]

[4426, 3837]

In [23]:
#from df_purdue_away, create a data frame with one row per game_id, with the following columns:
# - game_id: the game_id
# - away_scores: a list of scores for away_score for that game_id
# - home_scores: a list of scores for home_score for that game_id

df_purdue_away_scores = df_purdue_away.groupby(['game_id','half','away_team','home_team']).agg({'away_score': list, 'home_score': list}).reset_index()

#for each game_id,half pair, create a new column that is the array of the differences between the corresponding away_score and home_score values in the arrays
df_purdue_away_scores['score_diff'] = df_purdue_away_scores.apply(lambda row: [a - h for a, h in zip(row['away_score'], row['home_score'])], axis=1)
# Reduce score_diff arrays to include only changes in the score_diff
df_purdue_away_scores['score_diff_changes'] = df_purdue_away_scores['score_diff'].apply(lambda x: [v for i, v in enumerate(x) if i == 0 or v != x[i-1]])

# For each game_id,half pair, create a new column that is the array of the sums of the corresponding away_score and home_score values in the arrays
df_purdue_away_scores['score_sum'] = df_purdue_away_scores.apply(lambda row: [a + h for a, h in zip(row['away_score'], row['home_score'])], axis=1)
df_purdue_away_scores['score_sum_changes'] = df_purdue_away_scores['score_sum'].apply(lambda x: [v for i, v in enumerate(x) if i == 0 or v != x[i-1]])

print(len(df_purdue_away_scores))
df_purdue_away_scores.head()

26


,game_id,half,away_team,home_team,away_score,home_score,score_diff,score_diff_changes,score_sum,score_sum_changes
0,401707980,1,Purdue Boilermakers,Marquette Golden Eagles,"[0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, ...","[0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, -1, -1, -...","[0, 2, -1, -4, -2, -4, -1, 2, 1, -2, 0, -2, -4...","[0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, ...","[0, 2, 5, 8, 10, 12, 15, 18, 19, 22, 24, 26, 2..."
1,401707980,2,Purdue Boilermakers,Marquette Golden Eagles,"[28, 28, 28, 28, 28, 30, 30, 30, 30, 30, 30, 3...","[35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 3...","[-7, -7, -7, -7, -7, -5, -5, -5, -5, -5, -5, -...","[-7, -5, -3, -4, -5, -7, -5, -7, -4, -7, -9, -...","[63, 63, 63, 63, 63, 65, 65, 65, 65, 65, 65, 6...","[63, 65, 67, 68, 69, 71, 73, 75, 78, 81, 83, 8..."
2,401707982,1,Purdue Boilermakers,Auburn Tigers,"[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5, 5, 5, 5, ...","[2, 2, 5, 5, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, ...","[-2, 1, -2, -2, -2, -5, -5, -5, -5, -5, -5, -3...","[-2, 1, -2, -5, -3, -1, 1, -1, 1, -1, 0, -3, -...","[2, 5, 8, 8, 8, 11, 11, 11, 11, 11, 11, 13, 13...","[2, 5, 8, 11, 13, 15, 17, 19, 21, 23, 24, 27, ..."
3,401707982,2,Purdue Boilermakers,Auburn Tigers,"[26, 26, 26, 26, 26, 26, 26, 26, 26, 28, 28, 2...","[43, 43, 43, 45, 45, 45, 47, 47, 47, 47, 47, 4...","[-17, -17, -17, -19, -19, -19, -21, -21, -21, ...","[-17, -19, -21, -19, -20, -18, -20, -22, -21, ...","[69, 69, 69, 71, 71, 71, 73, 73, 73, 75, 75, 7...","[69, 71, 73, 75, 76, 78, 80, 82, 83, 84, 87, 8..."
4,401721310,1,Purdue Boilermakers,Penn State Nittany Lions,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 4, 4, 4, 4, 4, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 4, 4, 4, 4, 4, ...","[0, 1, 4, 5, 3, 1, 2, 3, 0, 2, 1, 0, -1, -2, -...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 4, 4, 4, 4, 4, ...","[0, 1, 4, 5, 7, 9, 10, 11, 14, 16, 17, 18, 19,..."


Below are a few games that a team played. Each row has three values: the game id, the half, and the sequence of score differences

game_id,half,score_diff_array
401707980,1,[0, 2, -1, -4, -2, -4, -1, 2, 1, -2, 0, -2, -4, -1, -3, -5, -6, -5, -3, -5, -7, -6, -9, -7, -10, -8, -10, -9, -8, -10, -8, -7]
401707980,2,[-7, -5, -3, -4, -5, -7, -5, -7, -4, -7, -9, -7, -6, -5, -8, -6, -8, -5, -6, -7, -8, -9, -8, -7, -6, -8, -9, -10, -12, -14, -13, -11, -13, -12, -14, -16, -13, -14, -15, -18, -19, -20, -18]
401707982,1,[-2, 1, -2, -5, -3, -1, 1, -1, 1, -1, 0, -3, -1, 0, -2, -4, -7, -5, -7, -8, -10, -12, -14, -12, -14, -16, -15, -14, -16, -17, -18, -16, -18, -15]
401707982,2,[-17, -19, -21, -19, -20, -18, -20, -22, -21, -20, -23, -21, -18, -20, -22, -24, -27, -29, -27, -28, -31, -33, -30, -29, -31, -29, -28, -27, -29, -26, -23, -26, -28, -25, -23, -26, -29, -28, -27, -26, -25, -24, -23, -21, -18]
401721310,1,[0, 1, 4, 5, 3, 1, 2, 3, 0, 2, 1, 0, -1, -2, -3, -4, -6, -7, -8, -9, -7, -10, -7, -9, -11, -9, -7, -9, -11, -13, -12, -13, -11, -12, -13, -15, -13, -15, -13, -16]
401721310,2,[-16, -14, -13, -14, -15, -18, -16, -14, -13, -15, -18, -19, -20, -18, -20, -21, -22, -24, -26, -25, -24, -27, -26, -25, -27, -25, -26, -27, -24, -21, -24, -25, -26, -24, -22, -20, -17, -18, -19, -20, -21, -22, -23, -20, -19, -18, -20, -18, -15, -16, -14, -11]
401721323,1,[0, -2, -4, -2, 0, 2, 4, 2, 4, 2, 0, 1, -1, 2, 5, 3, 5, 4, 2, 4, 2, 4, 6, 4, 6, 3, 4, 2, 1]
401721323,2,[1, 3, 1, -1, 2, 0, 1, 2, 0, 2, 5, 7, 6, 4, 6, 4, 6, 7, 4, 6, 9, 11, 9, 12, 13, 14, 15, 16, 15, 16, 17, 15, 12, 15, 14, 17, 14, 17, 15, 13, 16, 17, 19, 22, 21, 20]
401721343,1,[0, 2, 4, 3, 2, 4, 1, 3, 5, 7, 8, 5, 7, 6, 5, 7, 4, 6, 8, 6, 8, 6, 9, 10, 11, 9, 8, 7, 10, 7, 9]
401721343,2,[9, 12, 14, 12, 10, 12, 9, 6, 8, 10, 8, 10, 8, 5, 8, 10, 12, 14, 12, 14, 16, 14, 13, 14, 16, 14, 15, 16, 17, 19, 20, 19, 18]
401721358,1,[0, 2, 1, 3, 5, 3, 1, 3, 0, 2, 4, 1, -2, -3, -5, -6, -4, -6, -4, -3, -5, -7, -6, -4, -7, -10, -8]
401721358,2,[-6, -3, -1, -3, -2, -1, 1, -1, 0, 1, -1, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 9, 11, 14, 11, 13, 10, 12, 11, 10, 12, 13, 15, 14, 13, 11, 10, 8, 11, 13, 10, 11, 12, 11]
401721364,1,[0, 1, -1, -2, 0, -3, -1, 0, 1, 3, 2, 1, 3, 4, 3, 5, 3, 0, -2, -1, 0, -3, 0, 2, 4, 3, 2, 0, -2, 0, -1, -2, 0, 1, 2, 3, 2, 1, 0, -1]
401721364,2,[-3, -1, 1, 3, 6, 7, 8, 5, 6, 4, 1, 3, 4, 5, 7, 5, 4, 6, 4, 6, 4, 5, 6, 8, 9, 10, 11, 8, 9, 6, 5, 4, 5, 6, 7, 8, 9, 7]
401721401,1,[0, -3, -1, -3, -2, -5, -7, -5, -3, -1, -3, -4, -2, -4, -2, 1, 3, 6, 3, 1, 2, 3, 2, 0, -2, -1, 0, -3, -1, -4, -6, -4, -2, -5, -4, -3, 0]
401721401,2,[-3, -1, 1, -1, 0, 3, 6, 4, 6, 9, 6, 9, 12, 9, 11, 9, 7, 8, 9, 7, 10, 7, 10, 9, 8, 6, 9, 7, 9, 6, 8, 6, 9, 11, 9, 8, 6, 5, 4, 3, 4, 5, 7, 8, 10, 9, 10, 9, 8, 9, 7, 8, 9]
401721420,1,[0, 2, 4, 7, 5, 6, 7, 5, 8, 6, 5, 4, 6, 3, 1, -1, 1, 3, 2, 1, 3, 5, 2, 4, 7, 5, 7, 5, 8, 10, 8, 11, 9, 8, 7, 4, 3, 4, 2]
401721420,2,[2, 4, 6, 3, 5, 6, 8, 10, 8, 6, 8, 9, 8, 6, 8, 6, 3, 4, 5, 4, 3, 1, 3, 0, -1, -2, 0, 2, 4, 2, 4, 1, -1, 1, -1, -2, -3, -4, -6, -3, -4, -5, -2]
401721436,1,[0, -2, 0, -3, -2, 0, -1, 1, -1, 1, -1, 0, 1, 3, 5, 3, 6, 4, 7, 5, 4, 3, 6, 5, 4, 6, 5, 4, 3, 2, 0, -2, -4, -1, -4, -2]
401721436,2,[-2, -4, -7, -9, -10, -7, -6, -8, -7, -6, -8, -6, -9, -11, -13, -11, -13, -11, -10, -12, -9, -6, -8, -7, -6, -8, -9, -6, -8, -6, -4, -6, -8, -9, -11, -12, -13, -11, -12, -9]
401721448,1,[0, 2, 0, -2, 0, -2, 0, -3, -5, -4, -3, -5, -3, -2, -4, -6, -4, -6, -3, -5, -3, 0, 2, 5, 7, 5, 8, 11, 9, 12]
401721448,2,[12, 9, 7, 5, 7, 5, 2, 0, 1, -1, -2, -4, -6, -8, -9, -11, -13, -11, -13, -11, -12, -13, -12, -13, -14, -15, -16, -14, -13, -12, -11, -10, -11, -13, -10, -12, -11, -14, -15, -16, -17, -18, -17, -15]
401721475,1,[0, 2, 1, 0, -3, -5, -3, -1, -3, -1, 0, 1, -1, -4, -7, -5, -4, -2, 1, 3, 1, 4, 6, 4, 6, 8, 6, 8, 6, 8, 7, 6, 9, 7, 5, 7, 5, 7, 9, 8, 7, 6, 5, 2]
401721475,2,[2, 4, 2, 4, 1, 3, 5, 7, 8, 9, 7, 5, 4, 6, 5, 6, 7, 8, 7, 6, 7, 8, 10, 8, 5, 3, 0, 3, 2, -1, -4, -2, 1, -1, 1, 3, 5, 3, 2, 4, 1, 0, -3, -4, -5, -4, -5, -6, -7, -8]

Now. Below is the row for the first half of a game. predict the row for the second half.

401743939,1,[0, 2, 4, 6, 4, 1, -2, -3, -4, -6, -3, -4, -5, -4, -6, -4, -5, -6, -7, -6, -5, -8, -10, -8, -10, -7, -6, -5, -7, -4, -6, -4, -6, -4, -7, -9, -10, -8, -7, -6, -5, -6, -7, -4]

401743939,2, ...

Make sure that the second half you are to predict reflects a realistic progression based on the first half's trend and other game sequences.

**answer it returned** : 
401743939,2,[-6, -8, -5, -9, -7, -9, -12, -10, -13, -15, -13, -16, -18, -16, -19, -17, -20, -18, -21, -19, -17, -20, -22, -19, -17, -15, -18, -16, -14, -16, -18]

**the actual answer**:
401743939,2,[-4, -7, -9, -7, -6, -8, -10, -8, -11, -9, -7, -8, -9, -12, -9, -11, -9, -11, -13, -14, -15, -14, -13, -14, -13, -12, -10, -12, -11, -10, -12, -9, -11, -13, -15, -16, -13, -16, -15, -14, -16, -14, -16, -17, -18]



In [82]:
def process_team_games(team_name: str, df: pd.DataFrame):
    #select only those games that the Purdue Boilermakers played as the away_team
    df_team_away = df[df['away_team'].str.contains(team_name)]
    #from df_purdue_away, create a data frame with one row per game_id, with the following columns:
    # - game_id: the game_id
    # - away_scores: a list of scores for away_score for that game_id
    # - home_scores: a list of scores for home_score for that game_id

    df_team_away_scores = df_team_away.groupby(['game_id','half','away_team','home_team']).agg({'away_score': list, 'home_score': list}).reset_index()

    #for each game_id,half pair, create a new column that is the array of the differences between the corresponding away_score minus home_score values in the arrays
    df_team_away_scores['score_diff'] = df_team_away_scores.apply(lambda row: [a - h for a, h in zip(row['away_score'], row['home_score'])], axis=1)
    # Reduce score_diff arrays to include only changes in the score_diff
    df_team_away_scores['score_diff_changes'] = df_team_away_scores['score_diff'].apply(lambda x: [v for i, v in enumerate(x) if i == 0 or v != x[i-1]])

    # For each game_id,half pair, create a new column that is the array of the sums of the corresponding away_score and home_score values in the arrays
    df_team_away_scores['score_sum'] = df_team_away_scores.apply(lambda row: [a + h for a, h in zip(row['away_score'], row['home_score'])], axis=1)
    df_team_away_scores['score_sum_changes'] = df_team_away_scores['score_sum'].apply(lambda x: [v for i, v in enumerate(x) if i == 0 or v != x[i-1]])
    return df_team_away_scores


In [39]:
def write_score_diff_sum_files(file_name_stem: str,df: pd.DataFrame):
    #write out the score_diff and sum arrays, one per line, to separate files
    diff_output_file_path = file_name_stem + "DiffChanges.txt" #'../data/purdueAwayDiffChanges.txt'
    sum_output_file_path = file_name_stem + "SumChanges.txt" 

    with open(diff_output_file_path, 'w') as f:
        for row in df[['game_id','half','away_team','home_team','score_diff_changes']].itertuples(index=False):
            game, half, away_team,home_team, score_diff_c = row
            f.write(f"{game},{half},{away_team},{home_team},{score_diff_c}\n")

    with open(sum_output_file_path, 'w') as f:
        for row in df[['game_id','half','away_team','home_team','score_sum_changes']].itertuples(index=False):
            game, half, away_team,home_team, score_sum_c = row
            f.write(f"{game},{half},{away_team},{home_team},{score_sum_c}\n")


    print(f"score_diff/sum arrays written to {file_name_stem}")

In [85]:
# 'Purdue Boilermakers' and 'High Point Panthers'
def process_and_write_game(team1_name: str, team2_name: str, team1_file_stem: str,team2_file_stem:str,df_OG: pd.DataFrame):
    df_team1 = process_team_games(team1_name,df_OG)
    write_score_diff_sum_files(team1_file_stem,df_team1)

    df_team2 = process_team_games(team2_name,df_OG)
    write_score_diff_sum_files(team2_file_stem,df_team2)

def process_and_write(team1_name: str, path: str, df_OG: pd.DataFrame):
    df_team1 = process_team_games(team1_name,df_OG)
    team1_file_stem = path + team1_name + "Away"
    write_score_diff_sum_files(team1_file_stem,df_team1)

In [ ]:
process_and_write_game('Purdue Boilermakers', 'High Point Panthers', '../data/purdueAway','../data/highpointAway',df_OG)

score_diff/sum arrays written to ../data/purdueAway
score_diff/sum arrays written to ../data/highpointAway


---
I set up an agent in you.com with the following prompt:

You are trying to predict the outcome of an NCAA basketball game. More specifically, the entire sequence of score differentials between two team. You will be given two files, one for each team, representing all their respective away games. The files will have the same structure:

'game_id','half','away_team','home_team','score_diff_changes'

Thus, there are two lines for each game: one for the first half and one for the second.

Your job is to predict the outcome of the game between them at a neutral site, ie., as if they are both away teams.

(Show your Predicted Score Differential Pattern as a simple list of score differences (first_team - second_team).)

There is unlikely to be any 
- Historical game data between these specific teams
- performance data against opponents of similar caliber
- file accessibility for real-time analysis

Do the best you can despite these deficits. We realize that your answer is speculative.

---

Here is the return for the query below and files

- query: please predict the game between 'Purdue Boilermakers' and 'High Point Panthers'
- files: purdueAwayDiffChanges.txt, highpointAwayDiffChanges.txt

---

Based on analysis of historical away game patterns and current season metrics, here's the predicted outcome for Purdue vs High Point at a neutral site:
Predicted Final Score: Purdue 78 - High Point 71 (+7 margin)
Win Probability: 68% chance of Purdue victory 
Key Prediction Factors:
Historical Away Performance:
Purdue averages -5.3 final margin in away games despite 8.4 lead changes/game 
High Point shows stronger away performance (data not fully available) 
Both teams show significant score fluctuations: Purdue averages 7.1 max lead/-15.2 min deficit 
Current Season Metrics:
Scoring Margin: Purdue +6.8 vs High Point +13.4 
Offensive Efficiency: High Point 82.2 PPG (#18) vs Purdue 77.9 PPG (#62) 
Defensive Edge: High Point allows 68.8 PPG vs Purdue's 71.1 PPG 
Competition Adjustment:
+4.5 point adjustment for Purdue facing tougher Big Ten competition 
Predicted Score Differential Pattern:
Unable to render image

Game Flow Expectations:
Early back-and-forth with 6-8 lead changes
Purdue establishes 9-12 point lead by halftime
High Point rallies to within 4 points mid-2nd half
Purdue pulls away in final 8 minutes
Critical Matchup Factors:
Purdue's 3P shooting (38.5% #10) vs High Point's perimeter defense
Rebound battle: High Point's 33.6 RPG vs Purdue's 32.4 RPG 
Turnover margin: Both teams top-50 in low turnovers 
This prediction considers Purdue's power conference experience offsetting High Point's superior scoring metrics. The 7-point margin reflects both teams' tendencies for mid-game surges and late-game execution.

---


In [87]:
# write a loop thru thursday_teams that runs process_and_write for each team
p = '../data/'
for team in thursday_teams:
    process_and_write(team, p, df_OG)

score_diff/sum arrays written to ../data/Auburn TigersAway
score_diff/sum arrays written to ../data/Alabama State HornetsAway
score_diff/sum arrays written to ../data/Louisville CardinalsAway
score_diff/sum arrays written to ../data/Creighton BluejaysAway
score_diff/sum arrays written to ../data/Michigan WolverinesAway
score_diff/sum arrays written to ../data/UC San Diego TritonsAway
score_diff/sum arrays written to ../data/Texas A&M AggiesAway
score_diff/sum arrays written to ../data/Yale BulldogsAway
score_diff/sum arrays written to ../data/Missouri TigersAway
score_diff/sum arrays written to ../data/Drake BulldogsAway
score_diff/sum arrays written to ../data/Texas Tech Red RaidersAway
score_diff/sum arrays written to ../data/UNC Wilmington SeahawksAway
score_diff/sum arrays written to ../data/Kansas JayhawksAway
score_diff/sum arrays written to ../data/Arkansas RazorbacksAway
score_diff/sum arrays written to ../data/St. John's Red StormAway
score_diff/sum arrays written to ../data/O

In [127]:
# write a loop thru thursday_teams that runs process_and_write for each team
p = '../data/friday/'
for team in friday_teams:
    process_and_write(team, p, df_OG)

score_diff/sum arrays written to ../data/friday/Ole Miss RebelsAway
score_diff/sum arrays written to ../data/friday/North Carolina Tar HeelsAway
score_diff/sum arrays written to ../data/friday/Iowa State CyclonesAway
score_diff/sum arrays written to ../data/friday/Lipscomb BisonsAway
score_diff/sum arrays written to ../data/friday/Marquette Golden EaglesAway
score_diff/sum arrays written to ../data/friday/New Mexico LobosAway
score_diff/sum arrays written to ../data/friday/Michigan State SpartansAway
score_diff/sum arrays written to ../data/friday/Bryant BulldogsAway
score_diff/sum arrays written to ../data/friday/Florida GatorsAway
score_diff/sum arrays written to ../data/friday/Norfolk State SpartansAway
score_diff/sum arrays written to ../data/friday/UConn HuskiesAway
score_diff/sum arrays written to ../data/friday/Oklahoma SoonersAway
score_diff/sum arrays written to ../data/friday/Memphis TigersAway
score_diff/sum arrays written to ../data/friday/Colorado State RamsAway
score_diff

---

you.com agent: bball game spread sequence predictor

You are trying to predict the outcome of an NCAA basketball game. More specifically, the entire sequence of score differentials between two team. You will be given two files, one for each team, representing all their respective away games. The files will have the same structure:

'game_id','half','away_team','home_team','score_diff_changes'

Thus, there are two lines for each game: one for the first half and one for the second.

Your job is to predict the outcome of the game between them at a neutral site, ie., as if they are both away teams.

(Show your Predicted Score Differential Pattern as a simple list of score differences (first_team - second_team).)

There is unlikely to be any 
- Historical game data between these specific teams
- performance data against opponents of similar caliber
- file accessibility for real-time analysis

Do the best you can despite these deficits. We realize that your answer is speculative.

---


---

you.com agent: bball game score sum sequence predictor

You are trying to predict the outcome of an NCAA basketball game. More specifically, the entire sequence of score sums between two team. You will be given two files, one for each team, representing all their respective away games. The files will have the same structure:

'game_id','half','away_team','home_team','score_sum_changes'

Thus, there are two lines for each game: one for the first half and one for the second.

Your job is to predict the outcome(sum of the two teams' scores) of the first half (half=1) and the second half (half=2) between them at a neutral site, ie., as if they are both away teams.

(Show your Predicted Score Sum Pattern as a simple list of score differences (first_team - second_team).)

There is unlikely to be any 
- Historical game data between these specific teams
- performance data against opponents of similar caliber
- file accessibility for real-time analysis

Do the best you can despite these deficits. We realize that your answer is speculative.

---